In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from brain_observatory_qc.data_access import from_lims

In [2]:
zdrift_dir = Path(r'\\allen\programs\mindscope\workgroups\learning\ophys\zdrift')
plane_interval = 0.75
lamf_table_fn = zdrift_dir / 'multiplane_zdrift_lamf.pkl'
lamf_table = pd.read_pickle(lamf_table_fn)

# lamf_table.set_index('oeid',inplace=True,drop=True)
# # add mouse_id to it
# lamf_table.reset_index(drop=False,inplace=True)

# lamf_table['min_cc_episodes'] = lamf_table.apply(lambda x: np.min([x['cc_episodes'][0], x['cc_episodes'][-1]]), axis=1)
# lamf_table['min_cc_fl'] = lamf_table.apply(lambda x: np.min(x.cc_fl), axis=1)
# lamf_table['zdrift_episodes'] = lamf_table.apply(lambda x: plane_interval * (x.mpi_episodes[-1] - x.mpi_episodes[0]), axis=1)
# lamf_table['zdrift_fl'] = lamf_table.apply(lambda x: plane_interval * (x.mpi_fl[1] - x.mpi_fl[0]), axis=1)


In [5]:
lamf_table['ocid'] = lamf_table.oeid.apply(lambda x: from_lims.get_ophys_container_id_for_ophys_experiment_id(x))

In [23]:
def get_motion_correction_crop_xy_range(oeid):
    motion_df = from_lims.load_rigid_motion_transform(oeid)    
    max_y = np.ceil(max(motion_df.y.max(), 1)).astype(int)
    min_y = np.floor(min(motion_df.y.min(), 0)).astype(int)
    max_x = np.ceil(max(motion_df.x.max(), 1)).astype(int)
    min_x = np.floor(min(motion_df.x.min(), 0)).astype(int)
    range_y = [-min_y, -max_y]
    range_x = [-min_x, -max_x]
    return range_y, range_x

In [24]:
lamf_table[['range_y', 'range_x']] = lamf_table.oeid.apply(lambda x: get_motion_correction_crop_xy_range(x))

ValueError: Columns must be same length as key

In [25]:
range_y_all = []
range_x_all = []
for i, row in lamf_table.iterrows():
    oeid = row.oeid
    range_y, range_x = get_motion_correction_crop_xy_range(oeid)
    range_y_all.append(range_y)
    range_x_all.append(range_x)

In [30]:
len(range_x_all)

1032

In [32]:
lamf_table['range_y'] = range_y_all
lamf_table['range_x'] = range_x_all

In [33]:
lamf_table.to_pickle(lamf_table_fn)

In [13]:
import ophys_mfish_dev.utils as utils

ModuleNotFoundError: No module named 'ophys_mfish_dev'

In [21]:
range_x

[22, -9]